In [5]:
import numpy as np
import scipy.io as spio
import scipy.stats as stats
import math
from sortedcollections import SortedSet

def build_step_for_feature(image_matrix):
    move_step = []

    for i in range(image_matrix.shape[1]):
        a = SortedSet(image_matrix[:,i].tolist())
        move_step.append(a)
    
    return move_step


class Node:
    def __init__(self,feature,threshold,left,right,label):
        self.feature=feature
        self.threshold=threshold
        self.left=left
        self.right=right
        self.label=label

def calculate_max_label_count(labels):
    count_map={}
    for i in range(len(labels)):
        if not(labels[i] in count_map):
            count_map[labels[i]]=1
        else:
            count_map[labels[i]]=count_map[labels[i]]+1
            
     ##check this       
    return max(count_map.values())
        
def calculate_max_label(labels):
    count_map={}
    curr_max=0
    max_label=0
    for i in range(len(labels)):
        if not(labels[i] in count_map):
            count_map[labels[i]]=1
        else:
            count_map[labels[i]]=count_map[labels[i]]+1
        if(count_map[labels[i]]>curr_max):
            curr_max=count_map[labels[i]]
            max_label=labels[i]
     ##check this       
    return max_label

def entropy(labels):
    if labels is None or len(labels)==0:
        return 0;
    return (len(labels)-calculate_max_label_count(labels))*1.0*(1.0/len(labels))
    
def shuffle_and_split(data,partition):
    ran_order = np.arange(len(data['X']))
    np.random.shuffle(ran_order)
    ran_order_training = ran_order[:(int)(len(data['X'])*partition)] 
    ran_order_test = ran_order[(int)(len(data['X'])*partition):] 
    training_data = data['X'][ran_order_training]
    training_label = data['Y'][ran_order_training]
    test_data = data['X'][ran_order_test]
    test_label = data['Y'][ran_order_test]
    return [training_data,training_label,test_data,test_label]

def calculate_optimal_feature_threshold(feature_space,labels):
    opt_f=0
    opt_t=0
    max_uncert_red=0.0
    
    for f in range(feature_space.shape[1]):
        a=(max(feature_space[:,f])-min(feature_space[:,f]))/10
        b=[min(feature_space[:,f])+i*a for i in range(10)]
        for t in b:
            left_labels=[]
            right_labels=[]
            for i in range(len(feature_space)):
                if(feature_space[i,f]<t):
                    left_labels.append(labels[i])
                else:
                    right_labels.append(labels[i])
            if(len(left_labels)==0 or len(right_labels)==0):
                continue
            uncertainty_red = entropy(labels)-(len(left_labels)*1.0/len(labels))*entropy(left_labels) - (len(left_labels)*1.0/len(labels))*entropy(right_labels)
            if(uncertainty_red > max_uncert_red):
                opt_f=f
                opt_t=t
    
    return [opt_f,opt_t]

def build_tree(feature_space,labels,param,live_count):
    global tree_depth
    if len(feature_space)<=param or live_count > 10:
        return Node(None,None,None,None,calculate_max_label(labels))
    
    live_count += 1
    tree_depth = max(tree_depth,live_count)
    print live_count
    
    left_space=[]
    left_labels=[]
    right_space=[]
    right_labels=[]
    
    [f,t]=calculate_optimal_feature_threshold(feature_space,labels)
    print(f,t)
    if(f==0 and t== 0):
        return Node(None,None,None,None,calculate_max_label(labels))
    
    for i in range(len(feature_space)):
        if(feature_space[i][f]<t):
            
            left_space.append(feature_space[i])
            left_labels.append(labels[i])
        else:
            right_space.append(feature_space[i])
            right_labels.append(labels[i])
    
    left_node=build_tree(np.array(left_space),left_labels,param,live_count)
    right_node=build_tree(np.array(right_space),right_labels,param,live_count)
    
    return Node(f,t,left_node,right_node,None)
    
    ## find the optimal feature and threshold
    ## split on the optimal feature and threshold
    ## node.left=build_tree(left_space)
    ## node.right=build_tree(right_space)
    ## return node


class DecisionTree:
    
    def __init__(self,num_classes,root):
        self.num_classes=num_classes
        self.root=root

                

    def train_model(self,training_data,labels,param):
        print("------Pre-processing Data-----")
        for i in range(len(training_data)):
            if (i == 0):
                feature_mle_mu = training_data[i]
                self.dim = (training_data[i].shape)[0]
            else:
                feature_mle_mu += training_data[i]
            
        feature_mle_mu = feature_mle_mu * 1.0/len(training_data)
        
        for i in range(len(training_data)):
            x_minus_mu_feature = training_data[i]-feature_mle_mu
            if (i == 0):
                feature_mle_sigma = x_minus_mu_feature**2
            else:
                feature_mle_sigma += x_minus_mu_feature**2
                
        feature_mle_sigma = feature_mle_sigma * 1.0/len(training_data)
        
        keys = np.linspace(0,self.dim-1,self.dim)
        feature_var = list(zip(keys,feature_mle_sigma))
        
        feature_var_sorted = sorted(feature_var, key=lambda f: f[1], reverse = True)
        self.top200 = [int(e[0]) for e in feature_var_sorted[:200]]  
        print("------Done Pre-processing Data -----")
        # slice the data
        training_data = training_data[:,self.top200]

        self.root=build_tree(training_data,labels,param,0)

    def predict(self,x):
        prediction=None
        curr_node=self.root
        while prediction==None:
            if curr_node.label!= None:
                prediction=curr_node.label
            else:
                if(x[curr_node.feature]<curr_node.threshold):
                    curr_node=curr_node.left
                else:
                    curr_node=curr_node.right
        return prediction
                
                
                
                
                
        
    def testing_error(self,test_data,labels):
        misses=0
        test_data = test_data[:,self.top200]
        for i in range(len(test_data)):
            if(self.predict(test_data[i])!=labels[i]):
                misses=misses+1
        return (misses*1.0/len(test_data))
            
    
        ##Steps:
        ##1: return argmax P[X=x/Y=y]*P[Y=y]

        #Partition = 0.7 for 70-30 split
        
        
mat = spio.loadmat('hw1data.mat', squeeze_me=True)
data_split = shuffle_and_split(mat,0.7)
image_matrix = data_split[0]
move_step = build_step_for_feature(image_matrix)
tree_depth = 0
label_array= data_split[1]

decision_tree= DecisionTree(10,None)
#[f,t]= calculate_optimal_feature_threshold(image_matrix[:30],label_array[:30])
decision_tree.train_model(image_matrix,label_array,10)

#Training error
print(decision_tree.testing_error(data_split[0],data_split[1]))
#Test Error
print(decision_tree.testing_error(data_split[2],data_split[3]))

------Pre-processing Data-----
------Done Pre-processing Data -----
1
(140, 25)
2
(185, 25)
3
(162, 25)
4
(179, 50)
5
(198, 225)
6
(181, 25)
7
(188, 225)
8
(164, 225)
9
(133, 25)
10
(187, 171)
11
(173, 225)
10
(191, 225)
11
(191, 135)
7
(198, 180)
8
(197, 125)
9
(198, 81)
10
(198, 27)
11
(196, 200)
9
(197, 234)
10
(198, 108)
11
(197, 176)
10
(197, 245)
11
(196, 100)
5
(182, 100)
6
(180, 207)
7
(179, 171)
8
(186, 216)
9
(186, 180)
10
(179, 106)
11
(199, 225)
11
(199, 225)
8
(198, 36)
9
(199, 207)
10
(197, 216)
11
(197, 135)
6
(198, 225)
7
(182, 237)
8
(183, 225)
9
(185, 18)
10
(198, 180)
11
(198, 108)
8
(186, 225)
9
(186, 21)
10
(184, 125)
11
(193, 207)
11
(198, 189)
10
(199, 90)
11
(199, 45)
4
(199, 150)
5
(198, 225)
6
(191, 175)
7
(189, 125)
8
(199, 14)
9
(198, 180)
10
(198, 81)
11
(198, 63)
9
(199, 67)
10
(199, 43)
11
(199, 25)
11
(199, 61)
10
(199, 119)
11
(199, 98)
11
(199, 130)
8
(198, 171)
9
(189, 234)
10
(193, 225)
11
(192, 225)
10
(194, 200)
11
(193, 225)
7
(198, 180)
8
(196, 2

(197, 225)
11
(198, 225)
4
(195, 245)
5
(197, 200)
6
(195, 227)
7
(195, 218)
8
(199, 25)
9
(196, 207)
10
(196, 117)
11
(193, 108)
9
(199, 223)
8
(194, 50)
9
(197, 90)
10
(199, 225)
11
(198, 225)
9
(199, 50)
10
(198, 225)
11
(199, 27)
7
(198, 175)
8
(197, 162)
9
(195, 236)
10
(198, 90)
11
(192, 175)
10
(197, 63)
11
(197, 27)
8
(199, 75)
6
(199, 75)
7
(0, 0)
5
(195, 253)
6
(194, 175)
7
(192, 200)
8
(192, 114)
9
(197, 25)
10
(199, 225)
11
(198, 225)
10
(199, 150)
9
(192, 146)
10
(199, 75)
10
(199, 225)
11
(199, 198)
8
(192, 247)
9
(198, 50)
10
(194, 153)
11
(196, 225)
10
(198, 187)
11
(199, 225)
7
(199, 75)
8
(194, 246)
9
(199, 45)
10
(198, 225)
11
(198, 189)
9
(192, 225)
10
(199, 45)
11
(192, 126)
10
(192, 250)
11
(199, 45)
8
(199, 230)
9
(199, 142)
9
(198, 225)
10
(198, 198)
11
(198, 63)
10
(198, 246)
11
(197, 225)
6
(194, 150)
7
(192, 175)
8
(196, 25)
9
(196, 18)
10
(196, 9)
11
(192, 102)
9
(199, 50)
10
(197, 175)
11
(196, 227)
10
(199, 233)
8
(199, 25)
9
(199, 3)
10
(192, 247)
11
(198

In [2]:

print(decision_tree.testing_error(image_matrix,label_array))
print(decision_tree.testing_error(image_matrix[6000:],label_array[6000:]))


0.3552
0.501


In [26]:
print(decision_tree.testing_error(image_matrix[0:1200,],label_array[0:1200]))

0.095


In [11]:
decision_tree.train_model(image_matrix[0:1000,],label_array[0:1000],5)

TypeError: train_model() takes 3 positional arguments but 4 were given

In [ ]:
import numpy as np
import scipy.io as spio
import scipy.stats as stats
import math
import time
from sortedcollections import SortedSet

def build_step_for_feature(image_matrix):
    move_step = []

    for i in range(image_matrix.shape[1]):
        a = SortedSet(image_matrix[:,i].tolist())
        move_step.append(a)
    
    return move_step

class Node:
    def __init__(self,feature,threshold,left,right,label):
        self.feature=feature
        self.threshold=threshold
        self.left=left
        self.right=right
        self.label=label

def calculate_max_label_count(labels):
    count_map={}
    for i in range(len(labels)):
        if not(labels[i] in count_map):
            count_map[labels[i]]=1
        else:
            count_map[labels[i]]=count_map[labels[i]]+1
            
     ##check this       
    return max(count_map.values())
    
def calculate_max_label(labels):
    count_map={}
    curr_max=0
    max_label=0
    for i in range(len(labels)):
        if not(labels[i] in count_map):
            count_map[labels[i]]=1
        else:
            count_map[labels[i]]=count_map[labels[i]]+1
        if(count_map[labels[i]]>curr_max):
            curr_max=count_map[labels[i]]
            max_label=labels[i]
     ##check this       
    return max_label

def entropy(labels):
    if labels is None or len(labels)==0:
        return 0;
    return (len(labels)-calculate_max_label_count(labels))*1.0*(1.0/len(labels))
    

def calculate_optimal_feature_threshold(feature_space,labels):
    opt_f=0
    opt_t=0
    max_uncert_red=0.0
    
    for f in range(feature_space.shape[1]):
        #print(range(min(feature_space[:,f]),max(feature_space[:,f]),1))
        #for t in range(min(feature_space[:,f]),min(max(feature_space[:,f]),500),10):
        for t in move_step[f]:
            left_labels=[]
            right_labels=[]
            for i in range(len(feature_space)):
                if(feature_space[i,f]<t):
                    left_labels.append(labels[i])
                else:
                    right_labels.append(labels[i])
            if(len(left_labels)==0 or len(right_labels)==0):
                continue
            uncertainty_red = entropy(labels)-(len(left_labels)*1.0/len(labels))*entropy(left_labels) - (len(left_labels)*1.0/len(labels))*entropy(right_labels)
            if(uncertainty_red > max_uncert_red):
                opt_f=f
                opt_t=t
    return [opt_f,opt_t]

def build_tree(feature_space,labels,param,live_count):
    global tree_depth
    if len(feature_space)<=param or live_count > 9:
        return Node(None,None,None,None,calculate_max_label(labels))
    
    live_count += 1
    tree_depth = max(tree_depth,live_count)
    print tree_depth
    
    left_space=[]
    left_labels=[]
    right_space=[]
    right_labels=[]
    
    [f,t]=calculate_optimal_feature_threshold(feature_space,labels)
    print(f,t)
    
    for i in range(len(feature_space)):
        if(feature_space[i][f]<t):
            
            left_space.append(feature_space[i])
            left_labels.append(labels[i])
        else:
            right_space.append(feature_space[i])
            right_labels.append(labels[i])
    
    left_node=build_tree(np.array(left_space),left_labels,param,live_count)
    right_node=build_tree(np.array(right_space),right_labels,param,live_count)
    return Node(f,t,left_node,right_node,None)
    
    ## find the optimal feature and threshold
    ## split on the optimal feature and threshold
    ## node.left=build_tree(left_space)
    ## node.right=build_tree(right_space)
    ## return node


class DecisionTree:
    
    def __init__(self,num_classes,root):
        self.num_classes=num_classes
        self.root=root

                

    def train_model(self,training_data,labels):
        print("------Pre-processing Data-----")
        for i in range(len(training_data)):
            if (i == 0):
                feature_mle_mu = training_data[i]
                self.dim = (training_data[i].shape)[0]
            else:
                feature_mle_mu += training_data[i]
            
        feature_mle_mu = feature_mle_mu * 1.0/len(training_data)
        
        for i in range(len(training_data)):
            x_minus_mu_feature = training_data[i]-feature_mle_mu
            if (i == 0):
                feature_mle_sigma = x_minus_mu_feature**2
            else:
                feature_mle_sigma += x_minus_mu_feature**2
                
        feature_mle_sigma = feature_mle_sigma * 1.0/len(training_data)
        
        keys = np.linspace(0,self.dim-1,self.dim)
        feature_var = list(zip(keys,feature_mle_sigma))
        
        feature_var_sorted = sorted(feature_var, key=lambda f: f[1], reverse = True)
        self.top200 = [int(e[0]) for e in feature_var_sorted[:200]]  
        print("------Done Pre-processing Data -----")
        # slice the data
        training_data = training_data[:,self.top200]
        t0 = time.time()
        self.root=build_tree(training_data,labels,40,0)
        t1 = time.time()
        print ("Total time taken into building the tree: " + str(t1-t0))

    def predict(self,x):
        prediction=None
        curr_node=self.root
        while prediction==None:
            if curr_node.label!= None:
                prediction=curr_node.label
            else:
                if(x[curr_node.feature]<curr_node.threshold):
                    curr_node=curr_node.left
                else:
                    curr_node=curr_node.right
        return prediction
                        
    def testing_error(self,test_data,labels):
        misses=0
        test_data = test_data[:,self.top200]
        for i in range(len(test_data)):
            if(self.predict(test_data[i])!=labels[i]):
                misses=misses+1
        return (misses*1.0/len(test_data))
        ##Steps:
        ##1: return argmax P[X=x/Y=y]*P[Y=y]

#Partition = 0.7 for 70-30 split
def shuffle_and_split(data,partition):
    ran_order = np.arange(len(data['X']))
    np.random.shuffle(ran_order)
    ran_order_training = ran_order[:(int)(len(data['X'])*partition)] 
    ran_order_test = ran_order[(int)(len(data['X'])*partition):] 
    training_data = data['X'][ran_order_training]
    training_label = data['Y'][ran_order_training]
    test_data = data['X'][ran_order_test]
    test_label = data['Y'][ran_order_test]
    return [training_data,training_label,test_data,test_label]
        
mat = spio.loadmat('hw1data.mat', squeeze_me=True)
data_split = shuffle_and_split(mat,0.1)
image_matrix = data_split[0]
move_step = build_step_for_feature(image_matrix)
tree_depth = 0
label_array= data_split[1]

decision_tree= DecisionTree(10,None)
#[f,t]= calculate_optimal_feature_threshold(image_matrix[:30],label_array[:30])
decision_tree.train_model(image_matrix,label_array)

#Training error
print(decision_tree.testing_error(data_split[0],data_split[1]))
#Test Error
print(decision_tree.testing_error(data_split[2],data_split[3]))